In [1]:
import pandas as pd 
import gzip
import json
import ast
import textblob

In [63]:
df123 = pd.read_csv('df123.csv')
df13 = pd.read_csv('df13.csv')

ARCHIVOS A UTILIZAR DF1_GAMES_SIN_NULOS
DF2_REVIEWS
DF3_ITEMS

In [2]:
ruta_json1 = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\output_steam_games.json'
ruta_json2 = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\australian_user_reviews.json'
ruta_json3 = r'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\australian_users_items.json'



In [3]:
#data = pd.read_json(ruta_json)
df1_games = pd.read_json(ruta_json1, lines= True)
df1_games.to_csv('output_steam_games.csv', index=False)
print(df1_games.head(2))



  publisher genres app_name title   url release_date  tags reviews_url specs  \
0      None   None     None  None  None         None  None        None  None   
1      None   None     None  None  None         None  None        None  None   

  price  early_access  id developer  
0  None           NaN NaN      None  
1  None           NaN NaN      None  


In [9]:
#Valores nulos de cada columna
df1_games.isna().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [4]:
# Convertir la columna 'price' a números y establecer los valores no numéricos como 0
df1_games['price'] = pd.to_numeric(df1_games['price'], errors='coerce').fillna(0)

# Mostrar los primeros registros del DataFrame con los cambios
#print(df1_games_sin_nulos.head(2))

In [5]:

# Convertir la columna 'release_date' a tipo datetime
df1_games['release_date'] = pd.to_datetime(df1_games['release_date'], errors='coerce')

# Convertir la columna 'discount_price' a tipo float
df1_games['price'] = df1_games['price'].astype(float)

# Convertir la columna 'early_access' a tipo booleano (True/False)
df1_games['early_access'] = df1_games['early_access'].astype(bool)

# Convertir la columna 'metascore' a tipo int
#df1_games_sin_nulos['metascore'] = df1_games_sin_nulos['metascore'].astype(int)

# Mostrar los primeros registros del DataFrame con los cambios
print(df1_games.head(2))


  publisher genres app_name title   url release_date  tags reviews_url specs  \
0      None   None     None  None  None          NaT  None        None  None   
1      None   None     None  None  None          NaT  None        None  None   

   price  early_access  id developer  
0    0.0          True NaN      None  
1    0.0          True NaN      None  


In [6]:
linea_dicc = []
with open(ruta_json2, 'r', encoding='utf-8') as a:
    for num_linea, linea in enumerate(a, start=1):
            data = eval(linea)
            linea_dicc.append(data)
df2_reviews = pd.DataFrame(linea_dicc)
df2_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [296]:
df2_reviews.shape

(25799, 3)

In [7]:
# Crear un diccionario para almacenar los datos desanidados
df2_reviews_desanidado = {'funny': [], 'posted': [],'last_edited': [],'item_id': [],'helpful': [],'recommend': [],'review': []}

# Iterar a través de las filas de df2_reviews
for index, row in df2_reviews.iterrows():
    # Obtener los valores de las listas en la columna "reviews"
    for item in row['reviews']:
        # Desanidar y almacenar los valores en el diccionario
       #df2_reviews_desanidado['user_id'].append(row['user_id'])
        df2_reviews_desanidado['funny'].append(item['funny'])
        df2_reviews_desanidado['posted'].append(item['posted'])
        df2_reviews_desanidado['last_edited'].append(item['last_edited'])
        df2_reviews_desanidado['item_id'].append(item['item_id'])
        df2_reviews_desanidado['helpful'].append(item['helpful'])
        df2_reviews_desanidado['recommend'].append(item['recommend'])
        df2_reviews_desanidado['review'].append(item['review'])

# Crear un nuevo DataFrame a partir del diccionario desanidado
df2_reviews_desanidado = pd.DataFrame(df2_reviews_desanidado)

# Mostrar las primeras filas del DataFrame desanidado
df2_reviews_desanidado.head()


,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [342]:
#Ver duplicados
df2_reviews_desanidado[df2_reviews_desanidado["funny"].duplicated(keep=False)].sort_values(by="funny")

,funny,posted,last_edited,item_id,helpful,recommend,review


In [341]:
#Eliminar duplicados
df2_reviews_desanidado.drop_duplicates(subset="funny", keep="first", inplace=True)

In [8]:
# Eliminar la columna anidada "reviews" del DataFrame original
df2_reviews.drop('reviews', axis=1, inplace=True)

# Concatenar df2_reviews_desanidado con df2_reviews
df2_reviews = pd.concat([df2_reviews, df2_reviews_desanidado], axis=1)

# Mostrar el DataFrame completo
df2_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [12]:
linea_dicc3 = []
with open(ruta_json3, 'r', encoding='utf-8') as a:
    for num_linea, linea in enumerate(a, start=1):
            data = eval(linea)
            linea_dicc3.append(data)
df3_items = pd.DataFrame(linea_dicc3)
df3_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [13]:
# Crear un diccionario para almacenar los datos desanidados
df3_items_desanidado = { 'item_id': [], 'item_name': [], 'playtime_forever': [], 'playtime_2weeks': []}

# Iterar a través de las filas de df3_items
for index, row in df3_items.iterrows():
    # Obtener los valores de las listas en la columna "items"
    for item in row['items']:
        # Desanidar y almacenar los valores en el diccionario
        #df3_items_desanidado['user_id'].append(row['user_id'])
        df3_items_desanidado['item_id'].append(item.get('item_id', None))  # Usa None si 'item_id' está ausente
        df3_items_desanidado['item_name'].append(item.get('item_name', None))  # Usa None si 'item_name' está ausente
        df3_items_desanidado['playtime_forever'].append(item.get('playtime_forever', None))  # Usa None si 'playtime_forever' está ausente
        df3_items_desanidado['playtime_2weeks'].append(item.get('playtime_2weeks', None))  # Usa None si 'playtime_2weeks' está ausente

# Crear un nuevo DataFrame a partir del diccionario desanidado
df3_items_desanidado = pd.DataFrame(df3_items_desanidado)

# Mostrar las primeras filas del DataFrame desanidado
df3_items_desanidado.head()


,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0


In [14]:
# Eliminar la columna anidada "items" del DataFrame original
df3_items.drop('items', axis=1, inplace=True)

# Concatenar df2_reviews_desanidado con df2_reviews
df3_items = pd.concat([df3_items, df3_items_desanidado], axis=1)



In [15]:
# Mostrar el DataFrame completo
df3_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,20,Team Fortress Classic,0,0
2,evcentric,137.0,76561198007712555,http://steamcommunity.com/id/evcentric,30,Day of Defeat,7,0
3,Riot-Punch,328.0,76561197963445855,http://steamcommunity.com/id/Riot-Punch,40,Deathmatch Classic,0,0
4,doctr,541.0,76561198002099482,http://steamcommunity.com/id/doctr,50,Half-Life: Opposing Force,0,0


In [16]:
# Pasando a horas
df3_items_desanidado['playtime_forever'] = df3_items_desanidado['playtime_forever'] / 60


In [17]:
df3_items_desanidado.head()

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,0.100000,0
1,20,Team Fortress Classic,0.000000,0
2,30,Day of Defeat,0.116667,0
3,40,Deathmatch Classic,0.000000,0
4,50,Half-Life: Opposing Force,0.000000,0


In [18]:
df2_reviews.head()


,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted April 21, 2011.",,43110,No ratings yet,True,2
3,doctr,http://steamcommunity.com/id/doctr,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,2
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,0


In [9]:
#Analisis de sentimiento
from textblob import TextBlob

# Función para analizar el sentimiento y asignar un valor
def get_sentiment_score(df2_reviews):
    analysis = TextBlob(df2_reviews)
    sentiment = analysis.sentiment.polarity
    
    # Clasificar en base al sentimiento
    if sentiment > 0:
        return 2  # Positivo
    elif sentiment < 0:
        return 0  # Malo
    else:
        return 1  # Neutral
# Aplicar la función a cada reseña y crear la nueva columna 'sentiment_analysis'
df2_reviews['sentiment_analysis'] = df2_reviews['review'].apply(get_sentiment_score)

In [10]:
df2_reviews.drop('review', axis=1, inplace=True)

In [11]:
df2_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,2
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted April 21, 2011.",,43110,No ratings yet,True,2
3,doctr,http://steamcommunity.com/id/doctr,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,2
4,maplemage,http://steamcommunity.com/id/maplemage,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,0


In [19]:
# Extrae el año de 'release_date'
df1_games['release_year'] = df1_games['release_date'].dt.year

# Expande la columna 'genres'
df_exploded = df1_games.explode('genres')

# Selecciona solo las columnas 'genres', 'release_year' y cambia el nombre de 'id' a 'item_id'
df_final_games = df_exploded[['genres', 'release_year', 'id','title']].rename(columns={'id': 'item_id'})



In [23]:
df_final_games.head()


,genres,release_year,item_id,title
88310,Action,2018.0,761140.0,Lost Summoner Kitty
88311,Free to Play,2018.0,643980.0,Ironbound
88312,Casual,2017.0,670290.0,Real Pool 3D - Poolians
88313,Action,2017.0,767400.0,弹炸人2222
88315,Action,2018.0,772540.0,Battle Royale Trainer


In [21]:
#Eliminar nulos
df_final_games.dropna(inplace=True)

In [590]:
#Ver duplicados
df_final_games[df_final_games["item_id"].duplicated(keep=False)].sort_values(by="item_id")

,genres,release_year,item_id,title


In [22]:
#Eliminar duplicados
df_final_games.drop_duplicates(subset="item_id", keep="first", inplace=True)

In [24]:
# Agrupa por 'item_id' y suma 'playtime_forever'
df3_items_final = df3_items_desanidado.groupby('item_id')['playtime_forever'].sum().reset_index()

# Imprime el resultado
print(df3_items_final)


      item_id  playtime_forever
0          10     289766.916667
1         100       5199.983333
2       10000       1050.766667
3        1002         14.900000
4       10040          0.000000
...       ...               ...
10973   99890       2135.600000
10974    9990         35.166667
10975   99900     287660.083333
10976   99910       7111.333333
10977   99920         39.066667

[10978 rows x 2 columns]


In [25]:
#Eliminar nulos
df3_items_final.dropna(inplace=True)

In [582]:
#Ver duplicados
df3_items_final[df3_items_final["item_id"].duplicated(keep=False)].sort_values(by="item_id")

,item_id,playtime_forever


In [26]:
df3_items_final['item_id'] = df3_items_final['item_id'].astype(float)


In [27]:
df_final_games['item_id'] = df_final_games['item_id'].astype(float)


In [38]:
df2_reviews_final['item_id'] = df2_reviews_final['item_id'].astype(float)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_11452\3038610033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_reviews_final['item_id'] = df2_reviews_final['item_id'].astype(float)


In [39]:
df2_reviews_final['sentiment_analysis'] = df2_reviews_final['sentiment_analysis'].astype(float)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_11452\2922469777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_reviews_final['sentiment_analysis'] = df2_reviews_final['sentiment_analysis'].astype(float)


In [28]:
df3_items_final.head()

,item_id,playtime_forever
0,10.0,289766.916667
1,100.0,5199.983333
2,10000.0,1050.766667
3,1002.0,14.900000
4,10040.0,0.000000


In [29]:
# Une los DataFrames
df13 = pd.merge(df_final_games, df3_items_final, left_on='item_id', right_on='item_id')

# Imprime el resultado
print(df13)

        genres  release_year   item_id  \
0       Action        1997.0  282010.0   
1       Action        1998.0      70.0   
2     Strategy        2006.0    1640.0   
3     Strategy        2006.0    1630.0   
4       Action        2006.0    2400.0   
...        ...           ...       ...   
8622    Action        2003.0  259340.0   
8623    Action        2003.0      30.0   
8624  Strategy        2003.0  200980.0   
8625    Action        2004.0   13230.0   
8626    Action        2004.0      80.0   

                                                title  playtime_forever  
0                                Carmageddon Max Pack        155.316667  
1                                           Half-Life      44714.200000  
2                      Disciples II: Gallean's Return        456.616667  
3                     Disciples II: Rise of the Elves        351.850000  
4                              The Ship: Murder Party      17469.266667  
...                                               .

In [30]:
df13.head(10)

,genres,release_year,item_id,title,playtime_forever
0,Action,1997.0,282010.0,Carmageddon Max Pack,1.553167e+02
1,Action,1998.0,70.0,Half-Life,4.471420e+04
2,Strategy,2006.0,1640.0,Disciples II: Gallean's Return,4.566167e+02
3,Strategy,2006.0,1630.0,Disciples II: Rise of the Elves,3.518500e+02
4,Action,2006.0,2400.0,The Ship: Murder Party,1.746927e+04
5,Action,2005.0,3800.0,Advent Rising,2.281833e+02
6,Indie,2006.0,1520.0,DEFCON,3.690633e+03
7,Action,2006.0,2420.0,The Ship: Single Player,3.234133e+03
8,Indie,2006.0,4000.0,Garry's Mod,7.472777e+06
9,Racing,2006.0,4230.0,RACE - The WTCC Game,1.349183e+03


In [66]:
def PlayTimeGenre(genero):
    global df13  # Acceso a la variable global df13
    try:
        # Filtrar juegos por el género específico en el DataFrame unido
        filtered_games = df13[df13['genres'] == genero]

        if not filtered_games.empty:
            # Encontrar el año con más horas jugadas para el género dado
            year_with_most_hours_played = filtered_games.groupby('release_year')['playtime_forever'].sum().idxmax()

            return {"Año de lanzamiento con más horas jugadas para " + genero: int(year_with_most_hours_played)}
        else:
            return {"Error": "No se encontraron juegos para el género especificado."}
    except Exception as e:
        # Manejo de errores
        return {"Error": str(e)}

In [67]:
genero_buscado = "Action"  # Reemplaza "Action" con el género que desees buscar.
resultados = PlayTimeGenre(genero_buscado)

if "Error" in resultados:
    print(resultados)  # Manejo del caso de error
else:
    print(resultados)  # Año con más horas jugadas


{'Año de lanzamiento con más horas jugadas para Action': 2012}


In [33]:
# Selecciona solo la columna user_id y mantén el nombre de la columna
df2_reviews_final = df2_reviews[['user_id','item_id','recommend','sentiment_analysis']]


In [34]:
df2_reviews_final.head(3)

,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2
1,js41637,22200,True,2
2,evcentric,43110,True,2


In [35]:
#Eliminar nulos
df2_reviews_final.dropna(inplace=True)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_11452\2239958477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_reviews_final.dropna(inplace=True)


In [569]:
#Ver duplicados
df2_reviews_final[df2_reviews_final["item_id"].duplicated(keep=False)].sort_values(by="item_id")

,user_id,item_id,recommend,sentiment_analysis


In [36]:
#Eliminar duplicados
df2_reviews_final.drop_duplicates(subset="item_id", keep="first", inplace=True)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_11452\3632602750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_reviews_final.drop_duplicates(subset="item_id", keep="first", inplace=True)


In [40]:
# Une los 3 DataFrames
# Unir df123 con df2_reviews_final en 'item_id'
df123 = pd.merge(df13, df2_reviews_final, on='item_id')



In [61]:
df123.head(10)

,genres,release_year,item_id,title,playtime_forever,user_id,recommend,sentiment_analysis
0,Action,1997,282010.0,Carmageddon Max Pack,155.31666666666666,76561198076055973,True,1
1,Action,1998,70.0,Half-Life,44714.2,AppoGetsSuperAdmin,True,0
2,Action,2006,2400.0,The Ship: Murder Party,17469.266666666666,wiinga,True,2
3,Action,2005,3800.0,Advent Rising,228.18333333333334,daflyinskwirel,True,0
4,Indie,2006,1520.0,DEFCON,3690.633333333333,chriz2010,True,2
5,Action,2006,2420.0,The Ship: Single Player,3234.133333333333,76561197993160547,True,2
6,Indie,2006,4000.0,Garry's Mod,7472776.933333334,76561198061252210,True,1
7,Strategy,2006,6210.0,Vegas: Make It Big™,137.1,TaTayTaylor,True,2
8,Racing,2006,2990.0,FlatOut 2™,4498.016666666666,76561198061192570,True,0
9,Racing,2005,6220.0,FlatOut,709.6166666666667,lambor-sb,True,2


In [46]:
df123['release_year'] = df123['release_year'].astype(int)


In [48]:
df123['sentiment_analysis'] = df123['sentiment_analysis'].astype(int)


In [50]:
df123['title'] = df123['title'].astype(str)


In [56]:
df123['playtime_forever'] = df123['playtime_forever'].astype(str)


In [58]:
df123['user_id'] = df123['user_id'].astype(str)


In [60]:
df123['recommend'] = df123['recommend'].astype(bool)


In [62]:
df123.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2081 entries, 0 to 2080
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   genres              2081 non-null   object 
 1   release_year        2081 non-null   int32  
 2   item_id             2081 non-null   float64
 3   title               2081 non-null   object 
 4   playtime_forever    2081 non-null   object 
 5   user_id             2081 non-null   object 
 6   recommend           2081 non-null   bool   
 7   sentiment_analysis  2081 non-null   int32  
dtypes: bool(1), float64(1), int32(2), object(4)
memory usage: 99.7+ KB


In [68]:
def UserForGenre(genero):
    global df123  # Acceso a la variable global df123
    try:
        # Filtrar juegos por el género específico
        filtered_df = df123[df123['genres'] == genero]

        if not filtered_df.empty:
            # Encontrar al usuario que acumula más horas jugadas
            max_playtime_user = filtered_df.groupby('user_id')['playtime_forever'].sum().idxmax()

            # Calcular la acumulación de horas jugadas por año
            hours_played_by_year = filtered_df.groupby('release_year')['playtime_forever'].sum().reset_index()

            # Crear la lista de años y horas jugadas
            years = hours_played_by_year['release_year'].tolist()
            hours = hours_played_by_year['playtime_forever'].tolist()

            # Crear el diccionario de resultados
            result = {
                "Usuario con más horas jugadas para " + genero: max_playtime_user,
                "Horas jugadas": [{"Año": year, "Horas": hour} for year, hour in zip(years, hours)]
            }
            return result
        else:
            return {"Error": "No se encontraron juegos para el género especificado."}
    except Exception as e:
        return {"Error": str(e)}

In [70]:
genero_buscado = "Action"  # Reemplaza "Action" con el género que desees buscar.

resultados = UserForGenre(genero_buscado)

if "Error" in resultados:
    print(resultados)  # Manejo del caso de error
else:
    # Mostrar el usuario con más horas jugadas
    print("Usuario con más horas jugadas para", genero_buscado + ":", resultados["Usuario con más horas jugadas para " + genero_buscado])

    # Mostrar la acumulación de horas jugadas por año
    print("Acumulación de horas jugadas por año:")
    for data in resultados["Horas jugadas"]:
        print(f"Año: {data['Año']}, Horas: {data['Horas']}")


Usuario con más horas jugadas para Action: Fr0stedLine
Acumulación de horas jugadas por año:
Año: 1990.0, Horas: 311.9666666666667
Año: 1994.0, Horas: 1177.75
Año: 1995.0, Horas: 2581.9833333333336
Año: 1996.0, Horas: 354.2
Año: 1997.0, Horas: 2369.2
Año: 1998.0, Horas: 46954.566666666666
Año: 1999.0, Horas: 44801.583333333336
Año: 2000.0, Horas: 306506.26666666666
Año: 2001.0, Horas: 19489.466666666667
Año: 2002.0, Horas: 7146.933333333333
Año: 2003.0, Horas: 189660.93333333332
Año: 2004.0, Horas: 2184707.85
Año: 2005.0, Horas: 192722.83333333334
Año: 2006.0, Horas: 168924.18333333335
Año: 2007.0, Horas: 285004.05
Año: 2008.0, Horas: 561912.7833333333
Año: 2009.0, Horas: 2745279.3833333333
Año: 2010.0, Horas: 3015780.65
Año: 2011.0, Horas: 5564318.833333333
Año: 2012.0, Horas: 18231287.566666666
Año: 2013.0, Horas: 6964336.083333333
Año: 2014.0, Horas: 2741524.55
Año: 2015.0, Horas: 4132398.283333333
Año: 2016.0, Horas: 1560752.9166666667
Año: 2017.0, Horas: 2127526.7333333334


In [78]:
def UsersRecommend(anio: int):
    global df123  # Acceso a la variable global df123
    try:
        # Filtra el DataFrame para obtener solo las filas del año dado
        df_filtered = df123[df123['release_year'] == anio]

        if not df_filtered.empty:
            # Filtra las filas con recomendaciones positivas o neutrales
            df_filtered = df_filtered[df_filtered['recommend'] == 1.0]

            if not df_filtered.empty:
                # Ordena las filas por la cantidad de recomendaciones en orden descendente
                df_filtered = df_filtered.sort_values(by='recommend', ascending=False)

                # Selecciona las 3 primeras filas (los juegos más recomendados)
                top_3_recommendations = df_filtered.head(3)

                # Crea una lista con los resultados
                result = [{"Puesto " + str(i + 1): row['title']} for i, (_, row) in enumerate(top_3_recommendations.iterrows())]

                return result

        return {"Error": "No se encontraron juegos recomendados para el año especificado."}
    except Exception as e:
        return {"Error": str(e)}

In [79]:
año_buscado = 2006  # Reemplaza con el año que deseas buscar
resultados = UsersRecommend(año_buscado) 

if "Error" in resultados:
    print(resultados)  # Manejo del caso de error
else:
    print(resultados)  # Muestra los juegos más recomendados


[{'Puesto 1': 'The Ship: Murder Party'}, {'Puesto 2': '911: First Responders®'}, {'Puesto 3': 'Dark Messiah of Might &amp; Magic'}]


In [80]:
def UsersNotRecommend(anio: int):
    global df123  # Acceso a la variable global df123
    try:
        # Filtra el DataFrame para obtener solo las filas del año dado
        df_filtered = df123[df123['release_year'] == anio]

        if not df_filtered.empty:
            # Filtra las filas con recomendaciones negativas
            df_filtered = df_filtered[df_filtered['recommend'] == 0.0]

            if not df_filtered.empty:
                # Ordena las filas por la cantidad de recomendaciones en orden descendente
                df_filtered = df_filtered.sort_values(by='recommend', ascending=True)

                # Selecciona las 3 primeras filas (los juegos menos recomendados)
                top_3_not_recommendations = df_filtered.head(3)

                # Crea una lista con los resultados
                result = [{"Puesto " + str(i + 1): row['title']} for i, (_, row) in enumerate(top_3_not_recommendations.iterrows())]

                return result

        return {"Error": "No se encontraron juegos menos recomendados para el año especificado."}
    except Exception as e:
        return {"Error": str(e)}

In [81]:
año_buscado = 2006  # Reemplaza con el año que deseas buscar
resultados = UsersNotRecommend(año_buscado)

if "Error" in resultados:
    print(resultados)  # Manejo del caso de error
else:
    print(resultados)  # Muestra los juegos menos recomendados


[{'Puesto 1': 'The Stalin Subway'}, {'Puesto 2': 'Gothic® 3'}, {'Puesto 3': "Tom Clancy's Rainbow Six Lockdown™"}]


In [98]:
def sentiment_analysis(anio):
    try:
        # Filtra las filas del DataFrame para el año especificado
        filtered_data = df123[df123['release_year'] == anio]
        
        # Mapea los valores numéricos a las categorías deseadas
        sentiment_map = {0.0: 'Negative', 1.0: 'Neutral', 2.0: 'Positive'}
        filtered_data.loc[:, 'sentiment_analysis'] = filtered_data['sentiment_analysis'].map(sentiment_map)
        
        # Cuenta la cantidad de registros en cada categoría de análisis de sentimiento
        sentiment_counts = filtered_data['sentiment_analysis'].value_counts()
        
        # Convierte el resultado en un diccionario
        sentiment_dict = sentiment_counts.to_dict()
        
        return str(sentiment_dict)
    except Exception as e:
        # Manejo de errores
        return {"Error": str(e)}


In [99]:
año_deseado = 2012  # Reemplaza con el año que desees
resultados = sentiment_analysis(año_deseado)
print(resultados)


{'Positive': 76, 'Negative': 50, 'Neutral': 24}


In [89]:
recomendaciones = 5

In [94]:
def recomendacion_usuario(usuario_id, num_recomendaciones=5):
    global df123  # Acceso a la variable global df123

    # Filtrar las reseñas del usuario actual
    reseñas_usuario = df123[df123['user_id'] == usuario_id]

    # Encontrar usuarios similares
    usuarios_similares = df123[df123['user_id'] != usuario_id]  # Excluir al usuario actual
    usuarios_similares = usuarios_similares.groupby('user_id')['recommend'].mean().reset_index()

    # Ordenar usuarios similares por su similitud (recomendación promedio)
    usuarios_similares = usuarios_similares.sort_values(by='recommend', ascending=False)

    # Inicializar una lista para almacenar las recomendaciones
    recomendaciones = []

    # Iterar sobre los usuarios similares y encontrar juegos recomendados por ellos
    for i, row in usuarios_similares.iterrows():
        usuario_similar_id = row['user_id']
        juegos_recomendados = df123[(df123['user_id'] == usuario_similar_id) & (df123['recommend'] == 1)]['title'].tolist()

        # Evitar juegos que el usuario ya haya revisado
        juegos_recomendados = [juego for juego in juegos_recomendados if juego not in reseñas_usuario['title'].tolist()]

        recomendaciones.extend(juegos_recomendados)

        # Detener la búsqueda cuando se alcance el número deseado de recomendaciones
        if len(recomendaciones) >= num_recomendaciones:
            break

    # Tomar las primeras 'num_recomendaciones' recomendaciones
    recomendaciones = recomendaciones[:num_recomendaciones]

    return str(recomendaciones)


In [95]:
# Supongamos que tienes la función recomendacion_usuario definida en tu código

# Ejemplo de usuario_id
usuario_id = '76561198076055973'

# Llama a la función y obtén las recomendaciones
recomendaciones = recomendacion_usuario(usuario_id)

# Imprime las recomendaciones
print("Recomendaciones para el usuario", usuario_id, ":", recomendaciones)


Recomendaciones para el usuario 76561198076055973 : ['ORION: Prelude', 'Ford Racing 3', 'Blade Symphony', 'Realm of the Mad God', 'Momodora: Reverie Under The Moonlight']
